In [11]:
# This code reads in a lensing galaxy catalogue and appends to it 
# values of the mass over-density and tidal field tensor at the galaxy locations
# By Joachim Harnois-Deraps, 2021.

import numpy as np
import healpy as hp
#from scipy.interpolate import interp1d
import random
import input
from pathlib import Path


# The tomographic bin
tomobin = 5

print("**************************")
print("Processing tomo bin",tomobin)
print("**************************")

# Use the normal or smoothed tensor fields: '_V2' (0.1Mpc) or '_smooth' (0.5Mpc) or '_smooth_1Mpc' (1Mpc)
smooth_flag='_smooth_05Mpc'

# the number of galaxies per square arcminute to extract
gpam = 0.6

#SkySim5000:
#zfile=np.loadtxt("z2ts.txt",delimiter=',')
#snaplist = np.flip(zfile[1:58,1].astype(int))


#HACCY1:
nshells=51
zfile=np.loadtxt("z2ts_HACCY1_"+str(nshells)+".txt")
snaplist = np.flip(zfile[:,1].astype(int))

#BCM:
BCM_model='dmb'


n_slices = np.size(snaplist)


**************************
Processing tomo bin 5
**************************


In [12]:
#for plane in range(0,41):
for plane in range(0,n_slices):

    #d_smooth = '1' #'01', '05' or '1'
    d_smooth = '0.5' #'01', '05' or '1'
    
    print('starting analysis of plane %i' % plane)
            
    #SkySim5000:
    #V0
    #fname = '../../GalCat/StageIV_nz/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w.asc'
    #fname = '../../GalCat/SRD-Y1/random_pos/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w_z.asc'

    #V1
    #fname = '../../GalCat/StageIV_nz/V1/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w_linear_bias.asc'
    #fname = '../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w_linear_bias_bta1.asc'
    #fname = '../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w_linear_bias_bta2.asc'
    #fname = '../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_linear_bias_smooth'+d_smooth+'_bta2.asc'
    # note : the *planes* named bta1 are now bta2.

    #HACC/BCM:
    fname = '../../../BCM/'+BCM_model+'/GalCat/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_nshells51_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_z.asc'
    
    
    # check file exists:
    my_file = Path(fname)
    if my_file.is_file():
        # file exists
        print("Got file! Reading",fname)
        ra, dec, shear1, shear2, w, z = np.loadtxt(fname, unpack=True)
        
        #load mass maps and tidal tensor
        #SkySim5000:
        #fname = "../../density/density_map_"+str(snaplist[plane])+"_dens_allsky.npy"
        #fname = input.deltaDir+"density_map_"+str(snaplist[plane])+"_dens_allsky_smooth.npy"

        #HACC/BCM:
        fname = f"../../../BCM/{BCM_model}/fullsky_den/density_map_{plane:02d}_dens_allsky.npy"
        
        print("opening ", fname) 
        hpmap_density = np.load(fname)
        print("Got density file!")
        
        #SkySim5000:
        #fname = "../../tidalfield/tidal_field_map_"+str(snaplist[plane])+"_allsky"+smooth_flag+".npy"

        #HACC/BCM:
        fname = "../../../BCM/"+BCM_model+"/tidalfield/tidal_field_map_"+str(plane)+"_allsky"+smooth_flag+".npy"
        
        print("opening ", fname) 
        hpmap_tidal = np.load(fname)
        print("Got tidal file!")
        

        # Normalize the maps correctly:       
        mean_rho = (np.mean(hpmap_density)) # no factor of 8.0 since we have full sky data here
        hpmap_density /= mean_rho 
        hpmap_density -= 1
        
        # should express this in terms of shell thickness...
        Norm_empirical = 0.6525 
        
        hpmap_tidal *= Norm_empirical

        print("over density:",np.min(hpmap_density), np.max(hpmap_density), np.mean(hpmap_density))
        print("s11:",np.min(hpmap_tidal[:,0]), np.max(hpmap_tidal[:,0]), np.mean(hpmap_tidal[:,0]))
        print("s22:",np.min(hpmap_tidal[:,1]), np.max(hpmap_tidal[:,1]), np.mean(hpmap_tidal[:,1]))
        print("s12:",np.min(hpmap_tidal[:,2]), np.max(hpmap_tidal[:,2]), np.mean(hpmap_tidal[:,2]))


        
        # Interpolate maps at galaxy positions
        den_pix = hp.get_interp_val(hpmap_density, (90.-dec)*np.pi/180., ra*np.pi/180., nest=False )

        s11_pix = hp.get_interp_val(hpmap_tidal[:,0], (90.-dec)*np.pi/180., ra*np.pi/180., nest=False )
        s22_pix = hp.get_interp_val(hpmap_tidal[:,1], (90.-dec)*np.pi/180., ra*np.pi/180., nest=False )
        s12_pix = hp.get_interp_val(hpmap_tidal[:,2], (90.-dec)*np.pi/180., ra*np.pi/180., nest=False )



        # Save a shear catalogue in standard format to be read by TreeCorr
        #SkySim:
        #outname = '../../GalCat/SRD-Y1/random_pos/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_z_den_Sij'+smooth_flag+'.asc'
        #outname = input.GalDir+'StageIV_nz/V0/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_den_Sij.asc'
        #outname = '../../GalCat/StageIV_nz/V1/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias'+smooth_flag+'.asc'
        #outname = '../../GalCat/StageIV_nz/V1/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias'+smooth_flag+'_bta2.asc'
        #outname = '../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias'+smooth_flag+'_bta1.asc'
        #outname = '../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias'+smooth_flag+'_bta2.asc'
        #outname = '../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth'+d_smooth+'_'+smooth_flag+'_bta1.asc'
        #outname = '../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth'+d_smooth+'_'+smooth_flag+'_bta2_new.asc'

        #HACC/BCM:
        outname = '../../../BCM/'+BCM_model+'/GalCat/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_nshells51_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_z_den_Sij_'+smooth_flag+'.asc'

        print("Wrinting",outname)

        # given ra=np.degrees(phi), e2 DOES NOT NEED FLIPPING.
        #np.savetxt(outname, np.c_[ra_rand, dec_rand, shear1_pix_front*dz1 + shear1_pix_back*dz2, shear2_pix_front*dz1 + shear2_pix_back*dz2, np.ones(ngal_cur[plane]),spec_z[gal_cur],kappa_pix_front*dz1 + kappa_pix_back*dz2 ])
        np.savetxt(outname, np.c_[ra, dec, shear1, shear2, w,z, den_pix, s11_pix, s22_pix, s12_pix])



    else:
        print('found no file named '+fname)


print("Done!")

starting analysis of plane 0
Got file! Reading ../../../BCM/dmb/GalCat/GalCat_tomo5_plane0_nshells51_0.6GpAM_RA_Dec_g1_g2_w_z.asc
opening  ../../../BCM/dmb/fullsky_den/density_map_00_dens_allsky.npy
Got density file!
opening  ../../../BCM/dmb/tidalfield/tidal_field_map_0_allsky_smooth_05Mpc.npy
Got tidal file!
over density: -1.0 1548.4882 -4.409799e-06
s11: -1.5419073 169.36147 -0.0034284713
s22: -1.2130512 167.43471 0.0034201741
s12: -3.4908926 1.568504 5.2892705e-08
Wrinting ../../../BCM/dmb/GalCat/GalCat_tomo5_plane0_nshells51_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij__smooth_05Mpc.asc
starting analysis of plane 1
Got file! Reading ../../../BCM/dmb/GalCat/GalCat_tomo5_plane1_nshells51_0.6GpAM_RA_Dec_g1_g2_w_z.asc
opening  ../../../BCM/dmb/fullsky_den/density_map_01_dens_allsky.npy
Got density file!
opening  ../../../BCM/dmb/tidalfield/tidal_field_map_1_allsky_smooth_05Mpc.npy
Got tidal file!
over density: -1.0 2220.147 3.1883512e-06
s11: -8.037622 242.68913 -0.0039614732
s22: -7.843399 240.1

In [13]:
# Concatenate catalogues:
import os

print('Concatenating SkySim5000 plane catalogues')
#SkySim5000

#V0:
#os.system("for tomo in {1..1}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/*tomo$tomo*0.6GpAM*; done")
#os.system("for tomo in {1..1}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/*tomo$tomo*0.6GpAM* > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z.asc'; done")
#os.system("for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/random_pos/*tomo$tomo*0.6GpAM*_V2.asc; done")
#os.system("for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/random_pos/*tomo$tomo*0.6GpAM*_V2.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/random_pos/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_smooth0.1Mpc.asc'; done")
#os.system("for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/random_pos/*tomo$tomo*0.6GpAM*_smooth.asc; done")
#os.system("for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/random_pos/*tomo$tomo*0.6GpAM*_smooth.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/random_pos/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_smooth0.5Mpc.asc'; done")

#V1:
#os.system("for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/*tomo$tomo*0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias.asc; done")
#os.system("for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/*tomo$tomo*0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/GalCat_tomo$tomo'_All_0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias.asc'; done")
#os.system("for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/*tomo$tomo*0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc; done")
#os.system("for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/*tomo$tomo*0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/GalCat_tomo$tomo'_All_0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc'; done")
#os.system("for tomo in {2..2}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/*tomo$tomo*0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_1Mpc.asc; done")
#os.system("for tomo in {2..2}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/*tomo$tomo*0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_1Mpc.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/GalCat_tomo$tomo'_All_0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_1Mpc.asc'; done")
#os.system("for tomo in {1..5}; do echo 'working on tomo'$tomo; rm -v   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/*All_0.6GpAM*; done")
#os.system("for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc; done")
#os.system("for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc'; done")
#os.system("for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_bta2.asc; done")
#os.system("for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_bta2.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_bta2.asc'; done")
#os.system("let bta=2; for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_bta$bta.asc; done")
#os.system("let bta=2; for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_bta$bta.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_bta'$bta'_smooth0.5Mpc.asc'; done")
#os.system("let bta=2; for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_V2_bta$bta.asc; done")
#os.system("let bta=2; for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_V2_bta$bta.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_bta'$bta'_smooth0.1Mpc.asc'; done")
#os.system("let bta=1; smooth='1';for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth$smooth'__smooth_bta'$bta.asc; done")
#os.system("let bta=1; smooth='1';for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth$smooth'__smooth_bta'$bta.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth'$smooth'_bta'$bta'_smooth0.5Mpc.asc'; done")
#os.system("let bta=2; smooth='1';for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth$smooth'__smooth_bta'$bta.asc; done")
#os.system("let bta=2; smooth='1';for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth$smooth'__smooth_bta'$bta.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth'$smooth'_bta'$bta'_smooth0.5Mpc.asc'; done")

#HACK/BCM:
os.system("BCM='dmb'; for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/BCM/$BCM/GalCat/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij__smooth_05Mpc.asc; done")
os.system("BCM='dmb'; for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/BCM/$BCM/GalCat/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij__smooth_05Mpc.asc > /global/homes/j/jharno/IA-infusion/BCM/$BCM/GalCat/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_smooth0.5Mpc.asc'; done")


Concatenating SkySim5000 plane catalogues
working on tomo1
-rw-rw---- 1 jharno lsst      2279 Jan  7 11:17 /global/homes/j/jharno/IA-infusion/BCM/dmb/GalCat/GalCat_tomo1_plane25_nshells51_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij__smooth_05Mpc.asc
-rw-rw---- 1 jharno lsst     38482 Jan  7 11:17 /global/homes/j/jharno/IA-infusion/BCM/dmb/GalCat/GalCat_tomo1_plane26_nshells51_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij__smooth_05Mpc.asc
-rw-rw---- 1 jharno lsst    343754 Jan  7 11:17 /global/homes/j/jharno/IA-infusion/BCM/dmb/GalCat/GalCat_tomo1_plane27_nshells51_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij__smooth_05Mpc.asc
-rw-rw---- 1 jharno lsst   2006161 Jan  7 11:17 /global/homes/j/jharno/IA-infusion/BCM/dmb/GalCat/GalCat_tomo1_plane28_nshells51_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij__smooth_05Mpc.asc
-rw-rw---- 1 jharno lsst   8367220 Jan  7 11:17 /global/homes/j/jharno/IA-infusion/BCM/dmb/GalCat/GalCat_tomo1_plane29_nshells51_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij__smooth_05Mpc.asc
-rw-rw---- 1 jharno lsst  24509439 Jan  7

0

In [17]:
# Clean up
#HACK/BCM:
os.system("BCM='dmo'; rm -v  /global/homes/j/jharno/IA-infusion/BCM/$BCM/GalCat/*plane*")


removed '/global/homes/j/jharno/IA-infusion/BCM/dmo/GalCat/GalCat_tomo1_plane24_nshells51_0.6GpAM_RA_Dec_g1_g2_w_z.asc'
removed '/global/homes/j/jharno/IA-infusion/BCM/dmo/GalCat/GalCat_tomo1_plane24_nshells51_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij__smooth_05Mpc.asc'
removed '/global/homes/j/jharno/IA-infusion/BCM/dmo/GalCat/GalCat_tomo1_plane25_nshells51_0.6GpAM_RA_Dec_g1_g2_w_z.asc'
removed '/global/homes/j/jharno/IA-infusion/BCM/dmo/GalCat/GalCat_tomo1_plane25_nshells51_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij__smooth_05Mpc.asc'
removed '/global/homes/j/jharno/IA-infusion/BCM/dmo/GalCat/GalCat_tomo1_plane26_nshells51_0.6GpAM_RA_Dec_g1_g2_w_z.asc'
removed '/global/homes/j/jharno/IA-infusion/BCM/dmo/GalCat/GalCat_tomo1_plane26_nshells51_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij__smooth_05Mpc.asc'
removed '/global/homes/j/jharno/IA-infusion/BCM/dmo/GalCat/GalCat_tomo1_plane27_nshells51_0.6GpAM_RA_Dec_g1_g2_w_z.asc'
removed '/global/homes/j/jharno/IA-infusion/BCM/dmo/GalCat/GalCat_tomo1_plane27_nshells51_0.6G

0

In [ ]:
# For visualisation:
import matplotlib.pyplot as plt

plane = 56
Norm_empirical = 0.6525 

smooth_flag='_V2'
fname = "../../tidalfield/tidal_field_map_"+str(snaplist[plane])+"_allsky"+smooth_flag+".npy"

print("opening ", fname) 
hpmap_tidal = np.load(fname)
print("Got tidal file!")
#hpmap_tidal *= Norm_empirical

print("s11:",np.min(hpmap_tidal[:,0]), np.max(hpmap_tidal[:,0]), np.mean(hpmap_tidal[:,0]))
print("s22:",np.min(hpmap_tidal[:,1]), np.max(hpmap_tidal[:,1]), np.mean(hpmap_tidal[:,1]))
print("s12:",np.min(hpmap_tidal[:,2]), np.max(hpmap_tidal[:,2]), np.mean(hpmap_tidal[:,2]))

lonra = [0, 3]
latra = [0, 3]
hp.cartview(np.log(hpmap_tidal[:,0]), cbar=True, lonra=lonra, latra=latra,
        nest=False,
            title="plane "+np.str(plane),
            # min=-10, max=0,
            cmap=plt.cm.coolwarm,            
            norm=None, unit='$t_{11}$')
plt.show()

#--------------------
smooth_flag='_smooth'
fname = "../../tidalfield/tidal_field_map_"+str(snaplist[plane])+"_allsky"+smooth_flag+".npy"

print("opening ", fname) 
hpmap_tidal2 = np.load(fname)
print("Got tidal file!")
#hpmap_tidal *= Norm_empirical

print("s11:",np.min(hpmap_tidal2[:,0]), np.max(hpmap_tidal2[:,0]), np.mean(hpmap_tidal2[:,0]))
print("s22:",np.min(hpmap_tidal2[:,1]), np.max(hpmap_tidal2[:,1]), np.mean(hpmap_tidal2[:,1]))
print("s12:",np.min(hpmap_tidal2[:,2]), np.max(hpmap_tidal2[:,2]), np.mean(hpmap_tidal2[:,2]))

lonra = [0, 3]
latra = [0, 3]
hp.cartview(np.log(hpmap_tidal2[:,0]), cbar=True, lonra=lonra, latra=latra,
        nest=False,
            title="plane "+np.str(plane),
            # min=-10, max=0,
            cmap=plt.cm.coolwarm,            
            norm=None, unit='$t_{11}$')
plt.show()